In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import inception_resnet_v2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
!pip install matplotlib

In [ ]:
import cv2

fingerprint_image = cv2.imread('/kaggle/input/soco-fingerprint-female-and-male/SOCOFing/Train/Male/100__M_Left_index_finger.BMP', cv2.IMREAD_GRAYSCALE)

threshold_value = 127
binary_image = cv2.threshold(fingerprint_image, threshold_value, 255, cv2.THRESH_BINARY)
cv2.imshow('Original Fingerprint', fingerprint_image)
cv2.imshow('Binary Fingerprint', binary_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2

img = cv2.imread("/kaggle/input/fingerprint/1.jpg", 0)
print(img)  # Check if img is loaded correctly
cv2.imshow('enhanced_image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import PIL
import fingerprint_enhancer
import numpy as np

# Read the image using PIL
img = PIL.Image.open("/kaggle/input/fingerprint/4__M_Left_thumb_finger.BMP")

# Convert the image to grayscale
gray_img = img.convert("L")

# Resize the image
scaling_factor = 4  # Adjust the scaling factor as needed
resized_img = gray_img.resize((gray_img.width * scaling_factor, gray_img.height * scaling_factor))

# Convert resized PIL Image to NumPy array
resized_img_array = np.array(resized_img)

# Convert original grayscale PIL Image to NumPy array
original_img_array = np.array(gray_img)

# Enhance the fingerprint image
out = fingerprint_enhancer.enhance_Fingerprint(resized_img_array)

# Display the original grayscale image, resized image, and the enhanced image side by side
plt.figure(figsize=(15, 5))

# Original grayscale image
plt.subplot(1, 3, 1)
plt.imshow(original_img_array, cmap='gray')
plt.title('Original Grayscale Image')
plt.axis('off')

# Resized image
plt.subplot(1, 3, 2)
plt.imshow(resized_img_array, cmap='gray')
plt.title('Resized Image')
plt.axis('off')

# Enhanced image
plt.subplot(1, 3, 3)
plt.imshow(out, cmap='gray')
plt.title('Enhanced Image')
plt.axis('off')

plt.show()

In [ ]:
import cv2
import numpy as np

def enhance_fingerprint(image_path, output_path):
    # Baca gambar sidik jari
    fingerprint_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Menerapkan operasi morfologi untuk memperbaiki struktur sidik jari
    kernel = np.ones((5,5), np.uint8)
    opening = cv2.morphologyEx(fingerprint_image, cv2.MORPH_OPEN, kernel)
    
    # Menerapkan operasi top-hat untuk meningkatkan detail halus
    top_hat = cv2.morphologyEx(fingerprint_image, cv2.MORPH_TOPHAT, kernel)
    
    # Gabungkan gambar asli dengan hasil top-hat
    enhanced_image = cv2.add(fingerprint_image, top_hat)
    
    # Simpan gambar hasil
    cv2.imwrite(output_path, enhanced_image)
    
    print("Fingerprint enhancement complete. Enhanced image saved as", output_path)

# Contoh penggunaan
input_image_path = "/kaggle/input/soco-fingerprint-female-and-male/SOCOFing/ForTestingModel/Female/2__F_Left_index_finger.BMP"
output_image_path = "enhanced_fingerprint.jpg"
enhance_fingerprint(input_image_path, output_image_path)


In [ ]:
real_files = os.listdir(train_male_dir)
print(real_files[:10])

In [ ]:
%matplotlib inline

pic_index = 3


next_train = [os.path.join(train_male_dir, fname) 
                for fname in real_files[pic_index-2:pic_index]]


for i, img_path in enumerate(next_train):
  img = mpimg.imread(img_path)
  plt.imshow(img)
  plt.axis('Off')
  plt.show()
    


In [ ]:
def train_val_generators(training_dir, validation_dir, testing_dir):

    train_datagen = ImageDataGenerator(rescale=1./255.,
                                       rotation_range=40,
                                       width_shift_range=0.2,
                                       height_shift_range=0.2,
                                       shear_range=0.2,
                                       zoom_range=0.2,
                                       horizontal_flip=True,
                                       fill_mode='nearest')

    train_generator = train_datagen.flow_from_directory(directory=training_dir,
                                                        batch_size=64,
                                                        class_mode='categorical',
                                                        target_size=(224, 224))

    validation_datagen = ImageDataGenerator(rescale=1./255.)

    validation_generator = validation_datagen.flow_from_directory(directory=validation_dir,
                                                                  batch_size=64,
                                                                  class_mode='categorical',
                                                                  target_size=(224, 224))
    testing_datagen = ImageDataGenerator(rescale=1./255.)

    testing_generator = testing_datagen.flow_from_directory(directory=testing_dir,
                                                                  batch_size=64,
                                                                  class_mode='categorical',
                                                                  target_size=(224, 224))

    return train_generator, validation_generator, testing_generator

In [ ]:
training_dir = '/kaggle/input/soco-fingerprint-female-and-male/SOCOFing/Train'
validation_dir = '/kaggle/input/soco-fingerprint-female-and-male/SOCOFing/Validasi'
testing_dir = '/kaggle/input/soco-fingerprint-female-and-male/SOCOFing/Testing'
train_generator, validation_generator, testing_generator = train_val_generators(training_dir, validation_dir,testing_dir)

In [ ]:
from tensorflow.keras.applications import InceptionResNetV2
InceptionResNetV2_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in InceptionResNetV2_model.layers:
    layer.trainable = False

model = Sequential([
    InceptionResNetV2_model,
    GlobalAveragePooling2D(),
    Dense(2, activation='softmax')
])
print(model.summary())

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=0.001), 
    loss='binary_crossentropy', 
    metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

In [ ]:
true_classes = testing_generator.classes

class_labels = list(testing_generator.class_indices.keys())

predicted_classes = model.predict(testing_generator)
predicted_classes = np.argmax(predicted_classes, axis=1)

precision, recall, f1_score, support = precision_recall_fscore_support(
    true_classes, predicted_classes, average='weighted', labels=np.unique(predicted_classes))

accuracy = accuracy_score(true_classes, predicted_classes)

print('Precision:', precision)
print('Recall:', recall)
print('F1-Score:', f1_score)
print('Accuracy:', accuracy)

In [ ]:
cm = confusion_matrix(true_classes, predicted_classes)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Training', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Training', 'Validation'], loc='upper left')
plt.show()

In [ ]:
import numpy as np
import os
import shutil
from tensorflow.keras.utils import load_img, img_to_array
from IPython.display import Image
from PIL import Image as PILImage
import matplotlib.pyplot as plt
from PIL import Image

# Menggunakan PIL untuk membuka dan mengonversi gambar ke format RGB
img = Image.open("/kaggle/input/soco-fingerprint-female-and-male/SOCOFing/ForTestingModel/Female/2__F_Left_index_finger.BMP").convert("RGB")

# Mengubah gambar PIL ke array NumPy
img_array = np.array(img)

# Memeriksa bentuk array
print(img_array.shape)

# Mengunggah file gambar
uploaded_file = input("Masukkan path file gambar: ")
uploaded_file_name = os.path.basename(uploaded_file)

# Salin file gambar ke direktori saat ini
shutil.copyfile(uploaded_file, uploaded_file_name)

# Membaca gambar menggunakan PIL
img = PILImage.open(uploaded_file_name)
img = img.resize((224, 224))  # Mengubah ukuran gambar menjadi (150, 150)

plt.imshow(img)
plt.axis('off')
plt.show()

# Mengonversi gambar menjadi array dan melakukan normalisasi
x = img_to_array(img)
x /= 255
x = np.expand_dims(x, axis=0)

# Melakukan prediksi gambar
predictions = model.predict(x)

# Menentukan label kelas
class_labels = ['Female', 'Male']
predicted_class = class_labels[np.argmax(predictions)]
confidence = np.max(predictions) * 100

# Menampilkan hasil prediksi
print(f"{uploaded_file_name} terdeteksi sebagai: {predicted_class}")
print(f"Confidence: {confidence:.2f}%")

# Menghapus file yang diunggah
os.remove(uploaded_file_name)